In [202]:
import requests
from bs4 import BeautifulSoup
from requests_futures.sessions import FuturesSession
from ediblepickle import checkpoint
from collections import namedtuple
import os
import sys
import pandas as pd
import re
sys.setrecursionlimit(1000000)

In [4]:
URL_template = '''https://www.imdb.com/search/title/?title_type=feature&year={YEAR}-01-01,{YEAR}-12-31&view=simple&start={START}&ref_=adv_nxt'''



In [59]:
def GetListFromLink(url,year,start):
    url = url.format(YEAR=year,START=start)
    result = namedtuple('info',['name','year','popularity_rank_year','imdb_rate','imdb_link'])
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    parts = soup.select('div#main div.lister-list div.lister-item')
    res = []
    for part in parts:
        sub_part = part.find('div',class_='lister-col-wrapper')
        popularity = int(sub_part.find(class_='lister-item-index').text[:-1].replace(',',''))
        title_part = sub_part.find('div',class_='col-title')
        rate = sub_part.find('div',class_='col-imdb-rating').text.strip()
        if rate=='-':
            rate = None
        else:
            rate = float(rate)
        name= title_part.find('a').text.strip()
        link = '''https://www.imdb.com/'''+title_part.find('a')['href']
        year = title_part.find('span',class_='lister-item-year').text
        res.append(result(name,year,popularity,rate,link))
    return res

In [62]:
#df = pd.DataFrame(columns=['name','year','popularity','rate','link'])
for year in range(2010,2020):
    df = pd.DataFrame(columns=['name', 'year', 'popularity_rank_year', 'imdb_rate', 'imdb_link'])
    for start in range(1,5001,50):
        sys.stdout.write('process {}:{}/100'.format(year,int((start-1)/50)+1)+'\n')
        sys.stdout.flush()
        tmp = GetListFromLink(URL_template,year,start)
        for element in tmp:
            new_row = pd.Series(data={x:y for x,y in element._asdict().items()})
            df = df.append(new_row,ignore_index=True)
    df.to_csv('cache/{}.csv'.format(year))

process 2014:0/100
process 2014:1/100
process 2014:2/100
process 2014:3/100
process 2014:4/100
process 2014:5/100
process 2014:6/100
process 2014:7/100
process 2014:8/100
process 2014:9/100
process 2014:10/100
process 2014:11/100
process 2014:12/100
process 2014:13/100
process 2014:14/100
process 2014:15/100
process 2014:16/100
process 2014:17/100
process 2014:18/100
process 2014:19/100
process 2014:20/100
process 2014:21/100
process 2014:22/100
process 2014:23/100
process 2014:24/100
process 2014:25/100
process 2014:26/100
process 2014:27/100
process 2014:28/100
process 2014:29/100
process 2014:30/100
process 2014:31/100
process 2014:32/100
process 2014:33/100
process 2014:34/100
process 2014:35/100
process 2014:36/100
process 2014:37/100
process 2014:38/100
process 2014:39/100
process 2014:40/100
process 2014:41/100
process 2014:42/100
process 2014:43/100
process 2014:44/100
process 2014:45/100
process 2014:46/100
process 2014:47/100
process 2014:48/100
process 2014:49/100
process 20

In [275]:
def DetailFromPage(name,ID):
    res = namedtuple(name,['rank','Director','Writer','Producer','Editor','Casts','Budget','Country','origin_language','gross_USACanada'
                          'gross_WorldWide','Company','Runtime','Genres','NumberOfCritic','MovieMeter','Popularity','Awards','NumberOfReviews','NumberOfReviews_page'])

    url_imdb_pro = 'https://pro.imdb.com/title/tt1853728'
    url_imdb = 'https://www.imdb.com/title/tt1853728/'
    url_rotten = 'https://www.rottentomatoes.com/m/django_unchained_2012'  
    
    response_imdb = requests.get(url_imdb)
    response_imdb_pro = requests.get(url_imdb_pro)
    response_rotten = requests.get(url_rotten)
    
    soup_imdb = BeautifulSoup(response_imdb.text).find('div',id='content-2-wide')
    soup_imdb_pro = BeautifulSoup(response_imdb_pro.text)
    soup_rotten = BeautifulSoup(response_rotten.text)

    #Processing imdb page
    top_part = soup_imdb.find('div',id='main_top')
    res.rank = top_part.find('div',class_='title_wrapper').find('div',class_='subtext').text.split()[0]
    
    #processing plot_summary part
    plot_summary = top_part.find('div',class_='plot_summary')
    for element in plot_summary.select('h4.inline'):
        if 'Director' in element.text:
            res.Director = element.parent.find('a').text
        if 'Writer' in element.text:
            res.Writer =  element.parent.find('a').text
        if 'Star' in element.text:
            res.Producer = ' ; '.join([i.text for i in element.parent.select('a') if 'name' in i['href']])
    #release memoery
    del plot_summary
    
    
    titlebar = top_part.find('div',class_='titleReviewBar').findAll('div',class_='titleReviewBarItem')
    for element in titlebar:
        if 'Popularity' in element.split():
            res.Popularity = element.split()[1]
        if 'Reviews' in element.text.split():
            subtexts = element.findAll('a')
            for subtext in subtexts:
                if 'user' in subtext.text:
                    res.NumberOfReviews = subtext.text.split()[0]
                if 'critic' in subtext.text:
                    res.NumberOfCritic = subtext.text.split()[0]
    #release memory
    del titlebar
    
    
    bottom_part = soup_imdb.find('div',id='main_bottom') 
    #processing adwards
    Awards = bottom_part.find('div',id='titleAwardsRanks')
    tmp_awards= []
    if Awards.find('strong'):
        tmp_awards.append(Awards.find('strong').text.strip())
    for i in Awards.findAll('span',class_='awards-blurb'):
        tmp_awards.append(' '.join(i.text.split()))
    tmp_awards.append(Awards.find('span',class_='see-more').find('a')['href'])
    res.Adwards = ' ; '.join(tmp_awards)    
    #release memory
    del Awards
    
    #processing Genres
    Genres = bottom_part.find('div',id = 'titleStoryLine')
    for element in Genres.select('h4.inline'):
        if 'Genre' in element.text:
            res.Genres = ';'.join([i.text for i in element.parent.select('a')])
    #release memory
    del Genres
    
    #processing Cast
    Casts = bottom_part.find('div',id = 'titleCast').find('table',class_='cast_list')
    CastList = []
    for i in Casts.findAll('tr',class_= re.compile('even|odd')):
        CastList.append(i.find('td',class_=False).find('a').text.strip())
    res.Casts = ' ; '.join(CastList)
    #release memory
    del Casts
    
    #processing Details
    Details = bottom_part.find('div',id = 'titleDetails')
    Gross_tmp = []
    for element in Details.findAll('div',class_='txt-block'):
        if element.find('h4'):
            if 'Language' in element.find('h4').text:
                res.origin_language = ';'.join([i.textstrip() for i in element.findAll('a')])

            if 'Country' in element.find('h4').text:
                res.Country = ';'.join([i.text.strip() for i in element.findAll('a')])

            if 'Budget' in element.find('h4').text:
                res.Budget = element.find('h4').text.strip()

            Gross_tmp.append(element)

            ProductionList=[]
            if 'Production' in element.find('h4').text:
                for i in element.findChildren("a" , recursive=False):
                    ProductionList.append(i.text)
            res.Company = ';'.join(ProductionList)
            
            if 'Runtime' in element.find('h4').text:
                res.Runtime = int(element.find('time').text.split()[0])
            
    res.Editor =0
    res.MovieMeter = 0
    res.NumberOfReviews_page = 0
    
    return res
    

In [70]:
response = requests.get('https://www.imdb.com/title/tt1853728/')

In [276]:
DetailFromPage('asdf',4)

TypeError: 'NoneType' object is not callable

In [277]:
content = soup.find('div',id='content-2-wide').find('div',id='main_top')

In [281]:
tt = content.find('div',class_='titleReviewBar').findAll('div',class_='titleReviewBarItem')

In [282]:
for element in tt:
    if 'Popularity' in element.split():
        res.Popularity = element.split()[1]
    if 'Reviews' in element.text.split():
        subtexts = element.findAll('a')
        for subtext in subtexts:
            if 'user' in subtext.text:
                res.NumberOfReviews = subtext.text.split()[0]
            if 'critic' in subtext.text:
                res.NumberOfCritic = subtext.text.split()[0]


TypeError: 'NoneType' object is not callable

In [263]:
for i in a.findChildren("a" , recursive=False):
    print(i)

<a href="/company/co0150452"> The Weinstein Company</a>
<a href="/company/co0050868"> Columbia Pictures</a>


In [142]:
Producer

'Jamie Foxx ; Christoph Waltz ; Leonardo DiCaprio'

In [64]:
link1 = soup.select('div#main div.lister-list div.lister-item')[0]

In [88]:
new = link1.find('div',class_='lister-col-wrapper')

In [89]:
new.find('div',class_='col-title')

<div class="col-title">
<span class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<span title="Martin Scorsese (dir.), Leonardo DiCaprio, Jonah Hill">
<a href="/title/tt0993846/">The Wolf of Wall Street</a>
<span class="lister-item-year text-muted unbold">(2013)</span>
</span>
</span>
</div>

In [7]:
for year in range(2012,2018):
    for start in range(1,5001):
        GetListFromLink(URL_template,year,start)


0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
